# Abstract Extractor from Pubmed

This notebook was designed to autopopulate a MS SQL database with a fixed structure. Instructions and SQL code for creating this database can be found on the Wiki page for this project: https://github.com/KimSebek/Public-Health-Research-Information-Tool 

In [1]:
from Bio import Entrez
import time
import math
import pandas as pd
import pyodbc
from datetime import date, datetime

# Define Functions

In [2]:
class PubMed_Searcher:
    def __init__(self, email, retmode = 'xml'):
        self.email = email
        self.retmode = retmode
        Entrez.email = email
        
    #get count of how many records are returned
    def get_count(self, term):
        counthandle = Entrez.egquery(term = term)
        record = Entrez.read(counthandle)
        for row in record["eGQueryResult"]:
            if row['DbName'] == 'pubmed':
                rowcount = row['Count']
        counthandle.close()
        return rowcount
        
    #Used to interact with pubmed database
    def fetch_details(self, id_list):
        ids = ','.join(id_list)
        
        submitinterv = math.ceil(len(id_list)/10000)
        
        sleeptime = 5
        
        restartc = 0
        
        article_details = []
        
        for i in range(submitinterv):
            dethandle = Entrez.efetch(db='pubmed',
                                      retmode='xml',
                                      id=ids,
                                      retstart = restartc,
                                      retmax=10000)
            detailresults = Entrez.read(dethandle)
            article_details += detailresults['PubmedArticle']
            time.sleep(sleeptime)
            restartc += 10000
            dethandle.close()

        return article_details
    
    #function for retrieving and storing ids
    def pub_search(self,terms,sort='relevance',chunksize = 500,sleeptime = 5):
        
        submitinterv = math.ceil(int(self.get_count(term = terms))/chunksize)
        
        #append to idlist
        idlist = []

        retstartc = 0
        for i in range(submitinterv):
            handle = Entrez.esearch(db='pubmed', 
                                sort='relevance', 
                                retstart= retstartc,
                                retmax=chunksize,
                                retmode='xml', 
                                term=terms)
            idresults = Entrez.read(handle)
            idlist += idresults['IdList']
            time.sleep(sleeptime)
            retstartc += chunksize
            handle.close()
                
        return idlist

In [3]:
def get_article_df(articles):
    article_list = []
    for article in articles:
        
        ####### title ########
        title = article['MedlineCitation']['Article']['ArticleTitle']
        
        ####### pmid #########
        pmid = article['MedlineCitation']['PMID'].title()
        
        ####### journal ########
        journal = article['MedlineCitation']['Article']['Journal']['Title']
        
        
        ####### meshterms ##########
        
        #each meshterm will have its word and possibly some qualifiernames
        meshterm_list = []
        try:
            for meshterm in article['MedlineCitation']['MeshHeadingList']:
                #reset qualifiernames_list
                qualifiernames_list = []
                
                term = meshterm['DescriptorName'].title()
                if(len(meshterm['QualifierName']) != 0):
                    qualifiernames_list = [qn.title() for qn in meshterm['QualifierName']]
                
                meshterm_dict = {
                    'term' : term,
                    'qualifiernames' : qualifiernames_list
                }
                
                meshterm_list.append(meshterm_dict)
        except:
            meshterm_list = None
            
               
        ######## article date ###########
            
        if article['MedlineCitation']['Article']['ArticleDate']:
            year = article['MedlineCitation']['Article']['ArticleDate'][0]['Year']
            month = article['MedlineCitation']['Article']['ArticleDate'][0]['Month']
            day = article['MedlineCitation']['Article']['ArticleDate'][0]['Day']
            fulldate = '{0}/{1}/{2}'.format(month,day,year)
        if not article['MedlineCitation']['Article']['ArticleDate']:
            #sometimes the there is no DateCompleted either, so we will use a date given from the article history
            try:
                year = article['MedlineCitation']['DateCompleted']['Year']
                month = article['MedlineCitation']['DateCompleted']['Month']
                day = article['MedlineCitation']['DateCompleted']['Day']
                fulldate = '{0}/{1}/{2}'.format(month,day,year)
            except:            
                year = article['PubmedData']['History'][0]['Year']
                month = article['PubmedData']['History'][0]['Month']
                day = article['PubmedData']['History'][0]['Day']
                fulldate = '{0}/{1}/{2}'.format(month,day,year)
        
        
        ######### abstract ##########
         
        #each article's abstract will be stored as a dictionary. Each key being a different section of the abstract
        abstract_list = []
        try:
            try:
                #this means abstract is broken up by nlmcategory
                for abstractpart in article['MedlineCitation']['Article']['Abstract']['AbstractText']:
                    abstract_dict = {}
                    nlmcategory = abstractpart.attributes['NlmCategory'].lower()
                    label = abstractpart.attributes['Label'].lower()
                    text = abstractpart
                    abstract_dict['Label'] = label
                    abstract_dict['NlmCategory'] = nlmcategory
                    abstract_dict['text'] = text
                    
                    abstract_list.append(abstract_dict)
            except:
                #this means the abstract comes in as one whole chunk
                abstract_dict = {}
                wholetext = article['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
                abstract_dict['wholetext'] = wholetext
                
                abstract_list.append(abstract_dict)
        except:
            #this means there is no abstract at all
            abstract_list = None
            
            
        ############ authors ###############
        
        #each article will have a list of authors. Each author is represented by a dictionary (key values: fname, lname, affliation)
        auth_list = []
        #some articles don't have an author
        try:          
            #loop through authorlist for each article
            for author in article['MedlineCitation']['Article']['AuthorList']:
                
                #certain articles just have a collective name, for ex: CDC
                #those with a collective name don't have a forename or lastname, thus they are set to None
                try:
                    affiliation = author['CollectiveName']
                    lname = None
                    fname = None
                except:
                    try:
                        lname = author['LastName']
                    except:
                        lname = None

                    try:
                        fname = author['ForeName']
                    except:
                        fname = None

                    try:
                        affiliation = author['AffiliationInfo'][0]['Affiliation']
                    except:
                        affiliation = None
                
                auth_dict = {
                    'lname' : lname,
                    'fname' : fname,
                    'affiliation' : affiliation
                }
                
                auth_list.append(auth_dict)
        except:
            auth_list = None
            
            
        ######### optionalid01 ##########
        
        optionalid01 = None
        
        for elocationid in article['MedlineCitation']['Article']['ELocationID']:
            if elocationid.attributes['EIdType'] == 'doi':
                optionalid01 = elocationid    
                
            
        d = {
            'title': title,
            'pmid' : pmid,
            'author' : auth_list,
            'journal' : journal,
            'publishdate' : fulldate,
            'meshterms' : meshterm_list,
            'abstract' : abstract_list,
            'optionalid01' : optionalid01
        }
    
        article_list.append(d)
            
    return pd.DataFrame(article_list)

# search pubmed

In [12]:
####### query term #########
term = r'term'#insert your term here - this works best for a broad, generic keyword such as "diabetes"

####### email for pubmed #######
email = r'user@email.com'#insert your email here

In [13]:
Pubmed_obj = PubMed_Searcher(email)

In [14]:
pmidlist = Pubmed_obj.pub_search(terms=term,chunksize=10000)

In [15]:
articles = Pubmed_obj.fetch_details(pmidlist)

In [16]:
results_df = get_article_df(articles)

In [17]:
results_df.shape[0]

41242

# moving abstracts into the database

### establish connection

In [18]:
server = r'SERVER' #insert your server name here
database = r'DBASE' #insert your database name here

#trusted connection enables use of windows authentication(no need to enter your windows username/password)
cnxn = pyodbc.connect(driver='{SQL Server}',server=server,database=database,trusted_connection='yes')
cursor = cnxn.cursor()

### DataPull_ID table 

In [19]:
pulldate = date.today().strftime('%m/%d/%Y')
pullname = 'term' #insert your term here
pullquery = term
pulltype = 'keyword' #keyword/author
pullsource = 'Pubmed'.upper()
pullby = 'YOURNAME' #insert your name here 

query = """ insert into DataPull_ID (PullDate, PullName, PullQuery, PullType, PullSource, PullBy) 
            values (?, ?, ?, ?, ?, ?) """

args = (pulldate, pullname, pullquery, pulltype, pullsource, pullby)

cursor.execute(query, args)
#must commit in order to see it on sql server, if not sql server database won't load correctly
cnxn.commit()

### DataPull_Detail table

In [20]:
# gets PullID that was auto generated from query above
# do not run this if you are using refresh
pullid = cursor.execute("select @@IDENTITY").fetchall()[0][0].__str__()

In [ ]:
# get all AssociatedIDs already in the database
existing_ids = {associatedid[0] for associatedid in cursor.execute("select associatedid from DataPull_Detail").fetchall()}

note = None

query = """ insert into DataPull_Detail (PullID, AssociatedID, ValueStore, Note) values (?,?,?,?)"""

values_list = []

# check how many associatedIDs there are
for index,row in results_df.iterrows():
    
    associatedid = str(row['pmid'])
    
    # check if associatedid is already in the database
    if associatedid in existing_ids:
        valuestore = 'duplicate'
        # drop the row that already exists
        results_df.drop(index, inplace=True)
    else:
        valuestore = 'store'
        print('store')
    
    values_list.append((pullid,associatedid,valuestore,note))

cursor.executemany(query,values_list)
cnxn.commit()

### DataPull_Title table

In [22]:
query = """ insert into DataPull_Title (AssociatedID, Title, Journal, PublicationDate, OptionalID01, OptionalID02) values (?,?,?,?,?,?) """

values_list = []

for index,row in results_df.iterrows():
    associatedid = str(row['pmid'])
    title = row['title']
    journal = row['journal']
    pubdate = row['publishdate']
    id01 = row['optionalid01']
    
    values_list.append((associatedid, title, journal ,pubdate, id01, None))

cursor.executemany(query, values_list)
cnxn.commit()

### DataPull_Keyword table 

In [23]:
query = """ insert into DataPull_Keyword (AssociatedID, KeywordValue, Category1, Category2,
                    Category3, Category4, Category5) values (?,?,?,?,?,?,?)"""

values_list = []

for index,row in results_df.iterrows():
    associatedid = str(row['pmid'])
    
    if row['meshterms'] is not None:
        
        #each meshterm comes in as a dictionary (keys: qualifiernames & term)
        #qualifiernames value = a list of qualifernames
        #term value = the actual mesh term
        for word in row['meshterms']:
            keywordvalue = word['term']
            
            num_of_qualifiers = len(word['qualifiernames'])
            
            #some have more than 5 qualifier names, if that is the case, then make num_of_Nones to 0 
            #needed to fill up ? marks with NULLS
            
            if num_of_qualifiers > 5:
                num_of_Nones = 0
                word['qualifiernames'] = word['qualifiernames'][:5]
            else:
                num_of_Nones = 5 - num_of_qualifiers
                
            #if there are qualifier names, the list should not be 0
            if len(word['qualifiernames']) != 0:
                values_list.append(([associatedid,keywordvalue] + word['qualifiernames'] + [None]*num_of_Nones)) 
            else:
                values_list.append((associatedid, keywordvalue, None, None, None, None, None))

cursor.executemany(query,values_list)
cnxn.commit()  

### DataPull_Authors table

In [24]:
query = """ insert into DataPull_Author (AssociatedID, ForeName, LastName, Affiliation) values (?,?,?,?) """

values_list = []

for index,row in results_df.iterrows():
    associatedid = str(row['pmid'])
    auth_count = 0
    
    if row['author'] is not None:
        for auth in row['author']:
            auth_count += 1
            if auth_count < 4:
                values_list.append((associatedid, auth['fname'], auth['lname'], auth['affiliation']))
            else:
                break

cursor.executemany(query,values_list)
cnxn.commit()

### DataPull_Text table

In [25]:
query = """ insert into DataPull_text (associatedid, nlmcategory, label, abstracttext) values (?,?,?,?)"""

values_list = []

for index,row in results_df.iterrows():
    associatedid = str(row['pmid'])
    
    if row['abstract'] is not None:
        for part in row['abstract']:
            
            #check if the abstract is just one chunk
            try:
                values_list.append((associatedid, None, None, part['wholetext']))
            except:
                values_list.append((associatedid, part['NlmCategory'], part['Label'], part['text']))

cursor.executemany(query,values_list)
cnxn.commit()    

In [26]:
cnxn.close()